In [ ]:
!pip install snowflake-connector-python
!pip install tldextract
!pip install py2neo

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.site.gs.com/simple
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.site.gs.com/simple
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.site.gs.com/simple
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


In [ ]:
import requests
import pandas as pd
import numpy as np
import snowflake.connector
import os
from py2neo import Graph
from py2neo.bulk import merge_nodes, create_relationships, merge_relationships
from urllib.parse import urlparse

#[Environment Configs]
client_id = "d0afa05e7ab642acb0497f018b601b86"
client_secret = "ikdU3eOtXRmMR9oWlEnP0W0jAVZYNMBF2FUwsuwtHkZ37Oh4IZZkso3tFdsbfdmO"  
os.environ["HTTP_PROXY"] = "http://d161560-001.dc.gs.com:8899"
os.environ["HTTPS_PROXY"] = "http://d161560-001.dc.gs.com:8899"

#[Credentials]
host = "xplore.graph-x.site.gs.com"
port = "7687"
username = "neo4j"
password = "xplorer"
graph = Graph(host=host, port=port, auth=(username, password))
BATCHES = 100

### Libraries & Shared Code

In [ ]:
def get_oauth_tokens(client_id: str, client_secret: str):
    _IDFS_URL = "https://idfs.gs.com/as/token.oauth2"
    response = requests.post(
        url = _IDFS_URL,
        params = {"access_token_manager_id": "JwtSnowflake"},  # This must be set to JwtSnowflake to generate Snowflake-compatible OAuth Token
        data = {"grant_type": "client_credentials", "scope": "SESSION:ROLE-ANY"},
        auth = (client_id, client_secret),
    )
    response = response.json()
    if "access_token" not in response.keys():
        raise PermissionError(f"Not authorized to access OAuth Tokens: {response}")
    return response["access_token"]
  
def connect_snowflake(user: str, database: str, token: str) -> snowflake.connector.cursor:
    """Connect to Snowflake and return an authorized cursor. Below uses Catalyst connections
   
    :param user: Client ID from OAuth2 Authentication
    :param database: Name of database to connect to
    :param token: Generated OAuth Token. Please either generate and save to a tmp file in a separate thread OR use subprocess module to call the underlying subsystem and capture the output
    :return: Snowflake Cursor
    """
    snow_connector = snowflake.connector.connect(
        user=user,
        authenticator="oauth",
        account="sfamdprvtawseast1d01.goldman.us-east-1.aws.privatelink",
        role="AMD_CATALYST_DATA_RW",
        warehouse="AMD_CATALYST_RW",
        database=database,
        ocsp_fail_open=False,
        token=token,
    )
    cursor = snow_connector.cursor()
    return cursor
 
def disconnect(cursor: snowflake.connector.cursor) -> None:
    try:
        cursor.close()
    except:
        raise Exception("Failed to close cursor")   
        
oauth_token = get_oauth_tokens(client_id, client_secret)
cursor = connect_snowflake(client_id, "AMD_CATALYST_DB", oauth_token)

### SourceScrub Boardmembers

In [ ]:
sourcescrub_boardmembers_df = cursor.execute('SELECT PERSONID, COMPANYID, EMAIL, TITLE, FIRSTNAME, LASTNAME, LINKEDIN, STARTDATE, ENDDATE FROM AMD_CATALYST_DB.L2_NEW.SOURCESCRUB_BOARDMEMBERS WHERE LINKEDIN IS NOT NULL').fetch_pandas_all()
display(sourcescrub_boardmembers_df.head(5))

,PERSONID,COMPANYID,EMAIL,TITLE,FIRSTNAME,LASTNAME,LINKEDIN,STARTDATE,ENDDATE
0,sourcescrub-1561653,sourcescrub-614878,bblume@spacemakerai.com,Board Member,Ben,Blume,https://www.linkedin.com/in/benblume,2019-06-14,2020-11-14
1,sourcescrub-1561653,sourcescrub-608844,None,Board Member,Ben,Blume,https://www.linkedin.com/in/benblume,2021-02-14,None
2,sourcescrub-1561653,sourcescrub-245123,bblume@hingehealth.com,Board Member,Ben,Blume,https://www.linkedin.com/in/benblume,2019-01-14,None
3,sourcescrub-1561653,sourcescrub-1732426,None,Board Member,Ben,Blume,https://www.linkedin.com/in/benblume,2020-05-14,None
4,sourcescrub-1561653,sourcescrub-1713390,None,Board Member,Ben,Blume,https://www.linkedin.com/in/benblume,2019-03-01,None


              PERSONID            COMPANYID                    EMAIL  \
0  sourcescrub-1561653   sourcescrub-614878  bblume@spacemakerai.com   
1  sourcescrub-1561653   sourcescrub-608844                     None   
2  sourcescrub-1561653   sourcescrub-245123   bblume@hingehealth.com   
3  sourcescrub-1561653  sourcescrub-1732426                     None   
4  sourcescrub-1561653  sourcescrub-1713390                     None   

          TITLE FIRSTNAME LASTNAME                              LINKEDIN  \
0  Board Member       Ben    Blume  https://www.linkedin.com/in/benblume   
1  Board Member       Ben    Blume  https://www.linkedin.com/in/benblume   
2  Board Member       Ben    Blume  https://www.linkedin.com/in/benblume   
3  Board Member       Ben    Blume  https://www.linkedin.com/in/benblume   
4  Board Member       Ben    Blume  https://www.linkedin.com/in/benblume   

    STARTDATE     ENDDATE  
0  2019-06-14  2020-11-14  
1  2021-02-14        None  
2  2019-01-14        None 

In [ ]:
sourcescrub_boardmembers_df.rename(columns={'PERSONID': 'PERSON_ID', 'COMPANYID': 'COMPANY_ID', 'FIRSTNAME': 'FIRST_NAME',  'LASTNAME': 'LAST_NAME',  'STARTDATE': 'START_DATE', 'ENDDATE': 'END_DATE'}, inplace=True)
sourcescrub_boardmembers_df.dtypes

PERSON_ID     object
COMPANY_ID    object
EMAIL         object
TITLE         object
FIRST_NAME    object
LAST_NAME     object
LINKEDIN      object
START_DATE    object
END_DATE      object
dtype: object

In [ ]:
sourcescrub_boardmembers_df['LINKEDIN'] = sourcescrub_boardmembers_df['LINKEDIN'].apply(lambda x: (str(urlparse(x).netloc or '').lstrip('www.') + str(urlparse(x).path or '')))
sourcescrub_boardmembers_df.START_DATE = sourcescrub_boardmembers_df.START_DATE.fillna(0).astype('str').replace('0', np.nan)
sourcescrub_boardmembers_df.END_DATE = sourcescrub_boardmembers_df.END_DATE.fillna(0).astype('str').replace('0', np.nan)
sourcescrub_boardmembers_df.head(5)

,PERSON_ID,COMPANY_ID,EMAIL,TITLE,FIRST_NAME,LAST_NAME,LINKEDIN,START_DATE,END_DATE
0,sourcescrub-1561653,sourcescrub-614878,bblume@spacemakerai.com,Board Member,Ben,Blume,linkedin.com/in/benblume,2019-06-14,2020-11-14
1,sourcescrub-1561653,sourcescrub-608844,None,Board Member,Ben,Blume,linkedin.com/in/benblume,2021-02-14,NaN
2,sourcescrub-1561653,sourcescrub-245123,bblume@hingehealth.com,Board Member,Ben,Blume,linkedin.com/in/benblume,2019-01-14,NaN
3,sourcescrub-1561653,sourcescrub-1732426,None,Board Member,Ben,Blume,linkedin.com/in/benblume,2020-05-14,NaN
4,sourcescrub-1561653,sourcescrub-1713390,None,Board Member,Ben,Blume,linkedin.com/in/benblume,2019-03-01,NaN


             PERSON_ID           COMPANY_ID                    EMAIL  \
0  sourcescrub-1561653   sourcescrub-614878  bblume@spacemakerai.com   
1  sourcescrub-1561653   sourcescrub-608844                     None   
2  sourcescrub-1561653   sourcescrub-245123   bblume@hingehealth.com   
3  sourcescrub-1561653  sourcescrub-1732426                     None   
4  sourcescrub-1561653  sourcescrub-1713390                     None   

          TITLE FIRST_NAME LAST_NAME                  LINKEDIN  START_DATE  \
0  Board Member        Ben     Blume  linkedin.com/in/benblume  2019-06-14   
1  Board Member        Ben     Blume  linkedin.com/in/benblume  2021-02-14   
2  Board Member        Ben     Blume  linkedin.com/in/benblume  2019-01-14   
3  Board Member        Ben     Blume  linkedin.com/in/benblume  2020-05-14   
4  Board Member        Ben     Blume  linkedin.com/in/benblume  2019-03-01   

     END_DATE  
0  2020-11-14  
1         NaN  
2         NaN  
3         NaN  
4         NaN  

In [ ]:
sourcescrub_boardmembers_df['END_DATE']

0        2020-11-14
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
89417           NaN
89418           NaN
89419           NaN
89420           NaN
89421           NaN
Name: END_DATE, Length: 89422, dtype: object

In [ ]:
# Creating current board memebers
sourcescrub_curr_boardmembers_df = sourcescrub_boardmembers_df[sourcescrub_boardmembers_df['END_DATE'] .isna() == True]
print(sourcescrub_curr_boardmembers_df.shape)

# Creating sourcescrub_curr_boardmembers_df board members
sourcescrub_prev_boardmembers_df = sourcescrub_boardmembers_df[sourcescrub_boardmembers_df['END_DATE'].isna() == False]
print(sourcescrub_prev_boardmembers_df.shape)

(67526, 9)
(21896, 9)


### Ingesting Common SourceScrub Identifiers into Graph

In [ ]:
 ### Ingesting Common SourceScrub Identifiers into Graph
sourcescrub_curr_boardmembers_df_linkedin = np.array_split(sourcescrub_curr_boardmembers_df[sourcescrub_curr_boardmembers_df['LINKEDIN'].isna() == False][['LINKEDIN']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_curr_boardmembers_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "LinkedInURL"), "URL"),
    )

sourcescrub_prev_boardmembers_df_linkedin = np.array_split(sourcescrub_prev_boardmembers_df[sourcescrub_prev_boardmembers_df['LINKEDIN'].isna() == False][['LINKEDIN']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_prev_boardmembers_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "LinkedInURL"), "URL"),
    )

In [ ]:
sourcescrub_curr_boardmembers_df_email = np.array_split(sourcescrub_curr_boardmembers_df[sourcescrub_curr_boardmembers_df['EMAIL'].isna() == False][['EMAIL']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_curr_boardmembers_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("Identifier", "EmailURL"), "URL"),
    )

sourcescrub_prev_boardmembers_df_email = np.array_split(sourcescrub_prev_boardmembers_df[sourcescrub_prev_boardmembers_df['EMAIL'].isna() == False][['EMAIL']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in sourcescrub_prev_boardmembers_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "EmailURL"), "URL"),
    )

In [ ]:
for sourcescrub_boardmembers_df in [sourcescrub_prev_boardmembers_df, sourcescrub_curr_boardmembers_df]:
    for identifier in ['LINKEDIN', 'EMAIL']:
        sourcescrub_identifier_relationships = sourcescrub_boardmembers_df.loc[sourcescrub_boardmembers_df[identifier] != np.nan][['PERSON_ID', identifier]]
        sourcescrub_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(sourcescrub_boardmembers_df))
        df_list = np.array_split(sourcescrub_identifier_relationships, BATCHES)
        for chunk_df in df_list:
            merge_relationships(
                graph.auto(),
                chunk_df.values.tolist(),
                merge_key="LINKED_TO",
                start_node_key=("PERSON", "PERSON_ID"),
                end_node_key=("URLIdentifier", "URL"),
                keys=["WEIGHT"],
            )

#### SourceScrub Executives

In [ ]:
ss_exec_df = cursor.execute('SELECT * FROM AMD_CATALYST_DB.L2_NEW.SOURCESCRUB_EXECUTIVES WHERE LINKEDIN IS NOT NULL').fetch_pandas_all()
display(ss_exec_df.head(5))

,PERSONID,COMPANYID,EMAIL,TITLE,FIRSTNAME,LASTNAME,LINKEDIN,STARTDATE,ENDDATE
0,sourcescrub-1873836,sourcescrub-1883179,bogdan.yakovenko@serverbee.net,CEO,Bogdan,Yakovenko,https://www.linkedin.com/in/imxotep,2019-01-21,None
1,sourcescrub-1108552,sourcescrub-217812,smankiewicz@healthvana.com,CTO / VP Engineering,Sam,Mankiewicz,https://www.linkedin.com/in/sammankiewicz,2016-05-01,2018-10-01
2,sourcescrub-2372025,sourcescrub-2261938,ben@entek.be,Co-Owner,Ben,Van Ham,https://www.linkedin.com/in/benvanham,2012-03-01,None
3,sourcescrub-2759275,sourcescrub-1533681,gvenugopalan@rakwhitecement.ae,Director,Venugopalan,Ganapathy,https://www.linkedin.com/in/venugopalan-ganapa...,2015-03-01,None
4,sourcescrub-3879509,sourcescrub-13472744,None,Owner,Alex,Zavarzin,https://www.linkedin.com/in/alex-zavarzin-1879...,1997-01-01,None


              PERSONID             COMPANYID                           EMAIL  \
0  sourcescrub-1873836   sourcescrub-1883179  bogdan.yakovenko@serverbee.net   
1  sourcescrub-1108552    sourcescrub-217812      smankiewicz@healthvana.com   
2  sourcescrub-2372025   sourcescrub-2261938                    ben@entek.be   
3  sourcescrub-2759275   sourcescrub-1533681  gvenugopalan@rakwhitecement.ae   
4  sourcescrub-3879509  sourcescrub-13472744                            None   

                  TITLE    FIRSTNAME    LASTNAME  \
0                   CEO       Bogdan   Yakovenko   
1  CTO / VP Engineering          Sam  Mankiewicz   
2              Co-Owner          Ben     Van Ham   
3              Director  Venugopalan   Ganapathy   
4                 Owner         Alex    Zavarzin   

                                            LINKEDIN   STARTDATE     ENDDATE  
0                https://www.linkedin.com/in/imxotep  2019-01-21        None  
1          https://www.linkedin.com/in/sammankie

In [ ]:
ss_exec_df.rename(columns={'PERSONID': 'PERSON_ID', 'COMPANYID': 'COMPANY_ID', 'FIRSTNAME': 'FIRST_NAME',  'LASTNAME': 'LAST_NAME',  'STARTDATE': 'START_DATE', 'ENDDATE': 'END_DATE'}, inplace=True)

In [ ]:
ss_exec_df['LINKEDIN'] = ss_exec_df['LINKEDIN'].apply(lambda x: (str(urlparse(x).netloc or '').lstrip('www.') + str(urlparse(x).path or '')))
ss_exec_df.START_DATE = ss_exec_df.START_DATE.fillna(0).astype('str').replace('0', np.nan)
ss_exec_df.END_DATE = ss_exec_df.END_DATE.fillna(0).astype('str').replace('0', np.nan)

In [ ]:
ss_exec_df.head(5)

,PERSON_ID,COMPANY_ID,EMAIL,TITLE,FIRST_NAME,LAST_NAME,LINKEDIN,START_DATE,END_DATE
0,sourcescrub-1873836,sourcescrub-1883179,bogdan.yakovenko@serverbee.net,CEO,Bogdan,Yakovenko,linkedin.com/in/imxotep,2019-01-21,NaN
1,sourcescrub-1108552,sourcescrub-217812,smankiewicz@healthvana.com,CTO / VP Engineering,Sam,Mankiewicz,linkedin.com/in/sammankiewicz,2016-05-01,2018-10-01
2,sourcescrub-2372025,sourcescrub-2261938,ben@entek.be,Co-Owner,Ben,Van Ham,linkedin.com/in/benvanham,2012-03-01,NaN
3,sourcescrub-2759275,sourcescrub-1533681,gvenugopalan@rakwhitecement.ae,Director,Venugopalan,Ganapathy,linkedin.com/in/venugopalan-ganapathy-094a7443,2015-03-01,NaN
4,sourcescrub-3879509,sourcescrub-13472744,None,Owner,Alex,Zavarzin,linkedin.com/in/alex-zavarzin-1879a151,1997-01-01,NaN


             PERSON_ID            COMPANY_ID                           EMAIL  \
0  sourcescrub-1873836   sourcescrub-1883179  bogdan.yakovenko@serverbee.net   
1  sourcescrub-1108552    sourcescrub-217812      smankiewicz@healthvana.com   
2  sourcescrub-2372025   sourcescrub-2261938                    ben@entek.be   
3  sourcescrub-2759275   sourcescrub-1533681  gvenugopalan@rakwhitecement.ae   
4  sourcescrub-3879509  sourcescrub-13472744                            None   

                  TITLE   FIRST_NAME   LAST_NAME  \
0                   CEO       Bogdan   Yakovenko   
1  CTO / VP Engineering          Sam  Mankiewicz   
2              Co-Owner          Ben     Van Ham   
3              Director  Venugopalan   Ganapathy   
4                 Owner         Alex    Zavarzin   

                                         LINKEDIN  START_DATE    END_DATE  
0                         linkedin.com/in/imxotep  2019-01-21         NaN  
1                   linkedin.com/in/sammankiewicz  201

In [ ]:
# previous executives
ss_curr_exec_df = ss_exec_df[ss_exec_df['END_DATE'].isna() == True]
print(ss_curr_exec_df.shape)

# current executives
ss_prev_exec_df = ss_exec_df[ss_exec_df['END_DATE'].isna() == False]
print(ss_prev_exec_df.shape)

(2250083, 9)
(592672, 9)


In [ ]:
 ### Ingesting Common SourceScrub ExectutovesIdentifiers into Graph

for ss_exec_df in [ss_curr_exec_df, ss_prev_exec_df]:
    ss_exec_df_df_linkedin = np.array_split(ss_exec_df[ss_exec_df['LINKEDIN'].isna() == False][['LINKEDIN']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
    for chunk_df in sourcescrub_curr_boardmembers_df_linkedin:
        merge_nodes(
            graph.auto(),
            chunk_df,
            (("URLIdentifier", "LinkedInURL"), "URL"),
        )

In [ ]:
for ss_exec_df in [ss_prev_exec_df, ss_curr_exec_df]:
    for identifier in ['LINKEDIN', 'EMAIL']:
        sourcescrub_identifier_relationships = sourcescrub_boardmembers_df.loc[sourcescrub_boardmembers_df[identifier] != np.nan][['PERSON_ID', identifier]]
        sourcescrub_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(sourcescrub_boardmembers_df))
        df_list = np.array_split(sourcescrub_identifier_relationships, BATCHES)
        for chunk_df in df_list:
            merge_relationships(
                graph.auto(),
                chunk_df.values.tolist(),
                merge_key="LINKED_TO",
                start_node_key=("PERSON", "PERSON_ID"),
                end_node_key=("PERSON", "PERSON_ID"),
                keys=["WEIGHT"]
            ),
            merge_relationships(
                graph.auto(),
                chunk_df.values.tolist(),
                merge_key="LINKED_TO",
                start_node_key=("PERSON", "PERSON_ID"),
                end_node_key=("COMPANY", "COMPANY_ID"),
                keys=["WEIGHT"]
            )

### CrunchBase Jobs
#### 1. Current Jobs

In [ ]:
cb_jobs_df = cursor.execute('SELECT UUID, "JOB TYPE", NAME, "ORG NAME", "ORG UUID", PERMALINK, "PERSON NAME", "PERSON UUID"  FROM AMD_CATALYST_DB.L2.GRAPH_CRUNCHBASE_CURRENT_JOBS').fetch_pandas_all()
display(cb_jobs_df.head(5))

,UUID,JOB TYPE,NAME,ORG NAME,ORG UUID,PERMALINK,PERSON NAME,PERSON UUID
0,271cd67c-29e0-4981-80b9-624ead28de1a,executive,Geoffrey Goldberg Co-Founder & Chief Creative ...,Movers+Shakers,b5564a92-1a03-4e7c-924f-8eee7b7e9241,geoffrey-goldberg-b93b-executive-movers-shaker...,Geoffrey Goldberg,52c1da33-3fcd-4203-aad6-1ade3243b93b
1,97a9ea53-1d97-40c3-9bfc-3ed960502f41,employee,Anabel Roitz Business and product development ...,Youse Seguros,167c305a-4743-401c-8749-cd80e4e3a844,anabel-roitz-employee-youse-seguros--97a9ea53,Anabel Roitz,bfa09810-c845-494e-aa19-a85f759870f9
2,12d17236-98f4-4d42-8c84-09470886b3bc,executive,Luc Seraphin CEO @ Rambus,Rambus,b7fb95a3-cc31-1084-7258-b9b0869c0199,luc-seraphin-executive-rambus--12d17236,Luc Seraphin,da736ecc-2bb6-4252-bedc-00abcdc004ae
3,3adfd278-8a3e-429a-bcfb-7489dbc93e96,executive,Craig Zingerline Co-Founder @ Growth University,Growth University,c69717ab-4878-457c-8f9c-20b682a58226,craig-zingerline-executive-growth-university--...,Craig Zingerline,51abfa96-4415-8523-dfba-81fce13bf9d8
4,ac67f6fa-efff-7cc8-7e1b-c1a99e6bca61,executive,William Walczak Director @ Hiilite,Hiilite,9d0515a4-6af2-9ac9-da94-a652c4d7bb2f,william-walczak-executive-hiilite-creative-gro...,William Walczak,90ec2aac-0369-1b83-aae0-8bedb8f86fa4


                                   UUID   JOB TYPE  \
0  271cd67c-29e0-4981-80b9-624ead28de1a  executive   
1  97a9ea53-1d97-40c3-9bfc-3ed960502f41   employee   
2  12d17236-98f4-4d42-8c84-09470886b3bc  executive   
3  3adfd278-8a3e-429a-bcfb-7489dbc93e96  executive   
4  ac67f6fa-efff-7cc8-7e1b-c1a99e6bca61  executive   

                                                NAME           ORG NAME  \
0  Geoffrey Goldberg Co-Founder & Chief Creative ...     Movers+Shakers   
1  Anabel Roitz Business and product development ...      Youse Seguros   
2                          Luc Seraphin CEO @ Rambus             Rambus   
3    Craig Zingerline Co-Founder @ Growth University  Growth University   
4                 William Walczak Director @ Hiilite            Hiilite   

                               ORG UUID  \
0  b5564a92-1a03-4e7c-924f-8eee7b7e9241   
1  167c305a-4743-401c-8749-cd80e4e3a844   
2  b7fb95a3-cc31-1084-7258-b9b0869c0199   
3  c69717ab-4878-457c-8f9c-20b682a58226   
4  9d0515

In [ ]:
cb_jobs_df.rename(columns={
    'UUID': 'JOB_ID', 
    'JOB TYPE': 'JOB_TYPE', 
    'NAME': 'JOB_NAME',
    'ORG NAME': 'ORG_NAME',
    'ORG UUID': 'ORG_UUID', 
    'PERSON NAME': 'PERSON_NAME',
    'PERSON UUID': 'PERSON_UUID'
}, inplace=True)

cb_jobs_df['JOB_NAME'] =cb_jobs_df['JOB_NAME'].apply(lambda x: x.split("@")[0])
# cb_jobs_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
# cb_jobs_df.replace({None: ''}, inplace=True)

In [ ]:
cb_jobs_df.head()

,JOB_ID,JOB_TYPE,JOB_NAME,ORG_NAME,ORG_UUID,PERMALINK,PERSON_NAME,PERSON_UUID
0,271cd67c-29e0-4981-80b9-624ead28de1a,executive,Geoffrey Goldberg Co-Founder & Chief Creative ...,Movers+Shakers,b5564a92-1a03-4e7c-924f-8eee7b7e9241,geoffrey-goldberg-b93b-executive-movers-shaker...,Geoffrey Goldberg,52c1da33-3fcd-4203-aad6-1ade3243b93b
1,97a9ea53-1d97-40c3-9bfc-3ed960502f41,employee,Anabel Roitz Business and product development ...,Youse Seguros,167c305a-4743-401c-8749-cd80e4e3a844,anabel-roitz-employee-youse-seguros--97a9ea53,Anabel Roitz,bfa09810-c845-494e-aa19-a85f759870f9
2,12d17236-98f4-4d42-8c84-09470886b3bc,executive,Luc Seraphin CEO @ Rambus,Rambus,b7fb95a3-cc31-1084-7258-b9b0869c0199,luc-seraphin-executive-rambus--12d17236,Luc Seraphin,da736ecc-2bb6-4252-bedc-00abcdc004ae
3,3adfd278-8a3e-429a-bcfb-7489dbc93e96,executive,Craig Zingerline Co-Founder @ Growth University,Growth University,c69717ab-4878-457c-8f9c-20b682a58226,craig-zingerline-executive-growth-university--...,Craig Zingerline,51abfa96-4415-8523-dfba-81fce13bf9d8
4,ac67f6fa-efff-7cc8-7e1b-c1a99e6bca61,executive,William Walczak Director @ Hiilite,Hiilite,9d0515a4-6af2-9ac9-da94-a652c4d7bb2f,william-walczak-executive-hiilite-creative-gro...,William Walczak,90ec2aac-0369-1b83-aae0-8bedb8f86fa4


                                 JOB_ID   JOB_TYPE  \
0  271cd67c-29e0-4981-80b9-624ead28de1a  executive   
1  97a9ea53-1d97-40c3-9bfc-3ed960502f41   employee   
2  12d17236-98f4-4d42-8c84-09470886b3bc  executive   
3  3adfd278-8a3e-429a-bcfb-7489dbc93e96  executive   
4  ac67f6fa-efff-7cc8-7e1b-c1a99e6bca61  executive   

                                            JOB_NAME           ORG_NAME  \
0  Geoffrey Goldberg Co-Founder & Chief Creative ...     Movers+Shakers   
1  Anabel Roitz Business and product development ...      Youse Seguros   
2                          Luc Seraphin CEO @ Rambus             Rambus   
3    Craig Zingerline Co-Founder @ Growth University  Growth University   
4                 William Walczak Director @ Hiilite            Hiilite   

                               ORG_UUID  \
0  b5564a92-1a03-4e7c-924f-8eee7b7e9241   
1  167c305a-4743-401c-8749-cd80e4e3a844   
2  b7fb95a3-cc31-1084-7258-b9b0869c0199   
3  c69717ab-4878-457c-8f9c-20b682a58226   
4  9d0515

In [ ]:
cb_jobs_permalink_df = np.array_split(cb_jobs_df[cb_jobs_df['PERMALINK'].isna() == False][['PERMALINK']].rename(columns={'PERMALINK': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in cb_jobs_permalink_df:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "PERMALINK"), "URL"),
    )

In [ ]:
cb_jobs_jpn_df = np.array_split(cb_jobs_df[cb_jobs_df['JOB_NAME'].isna() == False][['JOB_NAME']].rename(columns={'JOB_TYPE': 'JOB_NAME'}).to_dict('records'), BATCHES)
for chunk_df in cb_jobs_permalink_df:
    merge_nodes(
        graph.auto(),
        chunk_df,
        ("JOB_TYPE", "JOB_NAME"),
    )

ValueError: ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
cb_jobs_org_df = np.array_split(cb_jobs_df[cb_jobs_df[['JOB_TYPE', 'JOB_NAME']].isna() == False][['JOB_TYPE', 'JOB_NAME']].rename(columns={'JOB_TYPE': 'JOB_NAME'}).to_dict('records'), BATCHES)
for chunk_df in cb_jobs_permalink_df:
    merge_nodes(
        graph.auto(),
        chunk_df,
        ("JOB_TYPE", "JOB_NAME"),
    )

In [ ]:

        cb_identifier_relationships = cb_jobs_df.loc[cb_jobs_df[identifier] != np.nan][['PERSON_ID', identifier]]
        cb_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(cb_jobs_df))
        df_list = np.array_split(cb_identifier_relationships, BATCHES)
        for chunk_df in df_list:
            merge_relationships(
                graph.auto(),
                chunk_df.values.tolist(),
                merge_key="LINKED_TO",
                start_node_key=("PERSON", "PERSON"),
                end_node_key=("", "JOB_ID"),
                keys=["WEIGHT"]
            ),
            merge_relationships(
                graph.auto(),
                chunk_df.values.tolist(),
                merge_key="LINKED_TO",
                start_node_key=("PERSON", "PERSON_ID"),
                end_node_key=("COMPANY", "COMPANY_ID"),
                keys=["WEIGHT"]
            ),
            merge_relationships(
                graph.auto(),
                chunk_df.values.tolist(),
                merge_key="LINKED_TO",
                start_node_key=("PERSON", "PERSON_ID"),
                end_node_key=("COMPANY", "COMPANY_ID"),
                keys=["WEIGHT"]
            )

In [ ]:
cb_jobs_df = np.array_split(crunchbase_organizations_df[cb_jobs_df['JOB_ID'].isna() == False][['JOB_ID']].rename(columns={'JOB': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_website:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "DomainURL"), "URL"),
    )

crunchbase_organizations_df_linkedin = np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['LINKEDIN'].isna() == False][['LINKEDIN']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "LinkedInURL"), "URL"),
    )
    
crunchbase_organizations_df_twitter = np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['TWITTER'].isna() == False][['TWITTER']].rename(columns={'TWITTER': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_twitter:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "TwitterURL"), "URL"),
    )

crunchbase_organizations_df_facebook =  np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['FACEBOOK'].isna() == False][['FACEBOOK']].rename(columns={'FACEBOOK': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_facebook:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "FacebookURL"), "URL"),
    )

crunchbase_organizations_df_crunchbase =  np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['CRUNCHBASE'].isna() == False][['CRUNCHBASE']].rename(columns={'CRUNCHBASE': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_crunchbase:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "CrunchbaseURL"), "URL"),
    )

In [ ]:
# crunchbase_organizations_df = cursor.execute('SELECT CRUNCHBASE_ORGANIZATION_UUID, ORGANIZATION_CITY, FACEBOOK_URL, DOMAIN, TWITTER_URL, FUNDING_ROUND_TYPE, FOUNDED_ON, EMPLOYEE_COUNT, CRUNCHBASE_GROUPS, ORGANIZATION_STATE_CODE, ORGANIZATION_NAME, LINKEDIN_URL, ORGANIZATION_COUNTRY_CODE, CRUNCHBASE_URL FROM L2_NEW.CRUNCHBASE_ORGANIZATIONS WHERE DOMAIN IS NOT NULL OR TWITTER_URL IS NOT NULL OR FACEBOOK_URL IS NOT NULL OR LINKEDIN_URL IS NOT NULL OR CRUNCHBASE_URL IS NOT NULL').fetch_pandas_all()


crunchbase_organizations_df.replace({None: ''}, inplace=True)

crunchbase_organizations_df['LINKEDIN'] = crunchbase_organizations_df['LINKEDIN'].apply(lambda x: (str(urlparse(x.replace('[', '').replace(']', '')).netloc or '') + str(urlparse(x.replace('[', '').replace(']', '')).path or '')).lstrip('www.'))
crunchbase_organizations_df['TWITTER'] = crunchbase_organizations_df['TWITTER'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))
crunchbase_organizations_df['FACEBOOK'] = crunchbase_organizations_df['FACEBOOK'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))
crunchbase_organizations_df['CRUNCHBASE'] = crunchbase_organizations_df['CRUNCHBASE'].apply(lambda x: (str(urlparse(x).netloc or '') + str(urlparse(x).path or '')).lstrip('www.'))

crunchbase_organizations_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
display(crunchbase_organizations_df.head())

In [ ]:
crunchbase_organizations_df_website = np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['DOMAIN'].isna() == False][['DOMAIN']].rename(columns={'DOMAIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_website:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "DomainURL"), "URL"),
    )

crunchbase_organizations_df_linkedin = np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['LINKEDIN'].isna() == False][['LINKEDIN']].rename(columns={'LINKEDIN': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_linkedin:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "LinkedInURL"), "URL"),
    )
    
crunchbase_organizations_df_twitter = np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['TWITTER'].isna() == False][['TWITTER']].rename(columns={'TWITTER': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_twitter:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "TwitterURL"), "URL"),
    )

crunchbase_organizations_df_facebook =  np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['FACEBOOK'].isna() == False][['FACEBOOK']].rename(columns={'FACEBOOK': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_facebook:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "FacebookURL"), "URL"),
    )

crunchbase_organizations_df_crunchbase =  np.array_split(crunchbase_organizations_df[crunchbase_organizations_df['CRUNCHBASE'].isna() == False][['CRUNCHBASE']].rename(columns={'CRUNCHBASE': 'URL'}).to_dict('records'), BATCHES)
for chunk_df in crunchbase_organizations_df_crunchbase:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("URLIdentifier", "CrunchbaseURL"), "URL"),
    )

In [ ]:
crunchbase_organizations = np.array_split([{k:v for k, v in x.items() if v == v} for x in crunchbase_organizations_df.to_dict('records')], BATCHES)
for chunk_df in crunchbase_organizations:
    merge_nodes(
        graph.auto(),
        chunk_df,
        (("Company", "CrunchbaseCompany"), "SOURCE_ID"),
    )

In [ ]:
for identifier in ['DOMAIN', 'LINKEDIN', 'TWITTER', 'FACEBOOK', 'CRUNCHBASE']:
    crunchbase_identifier_relationships = crunchbase_organizations_df.loc[crunchbase_organizations_df[identifier] != np.nan][['SOURCE_ID', identifier]]
    crunchbase_identifier_relationships.insert(1, 'WEIGHT', [[0.5]] * len(crunchbase_identifier_relationships))

    df_list = np.array_split(crunchbase_identifier_relationships, BATCHES)
    for chunk_df in df_list:
        merge_relationships(
            graph.auto(),
            chunk_df.values.tolist(),
            merge_key="LINKED_TO",
            start_node_key=("Company", "SOURCE_ID"),
            end_node_key=("URLIdentifier", "URL"),
            keys=["WEIGHT"],
        )

### CrunchBase Degress
create or replace view AMD_CATALYST_DB.L2.GRAPH_CRUNCHBASE_DEGREES(
	"CB URL",
	"COMPLETED ON",
	"CREATED AT",
	"CRUX INGESTION DT",
	"CRUX SUPPLIER IMPLIED DT",
	"DEGREE TYPE",
	"ENTITY DEF ID",
	"INSTITUTION NAME",
	"INSTITUTION UUID",
	"IS COMPLETED",
	NAME,
	PERMALINK,
	"PERSON NAME",
	"PERSON UUID",
	RANK,
	"STARTED ON",
	SUBJECT,
	TYPE,
	"UPDATED AT",
	UUID,
	CATALYST_LAST_UPDATED
) as
SELECT * FROM L1.CRUNCHBASE_DEGREES
QUALIFY
    ROW_NUMBER() OVER (PARTITION BY L1.CRUNCHBASE_DEGREES.UUID ORDER BY L1.CRUNCHBASE_DEGREES."CRUX INGESTION DT" DESC)=1;

### CrunchBase Events
create or replace view AMD_CATALYST_DB.L2.GRAPH_CRUNCHBASE_EVENTS(
	"APPEARANCE TYPE",
	"CB URL",
	"CREATED AT",
	"CRUX INGESTION DT",
	"CRUX SUPPLIER IMPLIED DT",
	"ENTITY DEF ID",
	"EVENT NAME",
	"EVENT UUID",
	NAME,
	"PARTICIPANT NAME",
	"PARTICIPANT TYPE",
	"PARTICIPANT UUID",
	PERMALINK,
	RANK,
	"SHORT DESCRIPTION",
	TYPE,
	"UPDATED AT",
	UUID,
	"CATALYST LAST UPDATED"
) as
  SELECT * FROM L1.CRUNCHBASE_EVENT_APPEARANCES
  QUALIFY
      ROW_NUMBER() OVER (PARTITION BY L1.CRUNCHBASE_EVENT_APPEARANCES.UUID ORDER BY L1.CRUNCHBASE_EVENT_APPEARANCES."CRUX INGESTION DT" DESC)=1;